# Notebook to test the module PageFinder

In [1]:
import pandas as pd
import sys
sys.path.append("../../")    # Add the path to the root directory (where we can find the folder narval/)

%load_ext autoreload
%autoreload 2 

from narval.pdfreader import PDFReader
from narval.pagefinder import PageFinder
from narval.utils import get_data_dir, FileSystem

/home/gefleury/GitHub/narval/.venv/lib/python3.12/site-packages/numpy/_core/getlimits.py:548: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


## Preliminary tasks (if needed)

### Build the keyword dataframe (from Malou's version)

In [10]:
question_all_df = pd.read_csv("../../data/input/question_all.csv", index_col=0)

question_all_df.head()

,question,mot,indic
0,Quel est le prix de service au m3 pour l'eau p...,prix,D102.0
1,Quel est le prix du service au m3 pour l'assin...,prix,D204.0
2,Quelle est la connaissance du réseau en eau po...,connaissance,P103.2B
3,Quel est le taux de connaissance du réseau en ...,connaissance,P202.2B
4,Quel est le taux de renouvellement des réseaux...,renouvellement,P107.2


In [9]:
indicator_df = pd.read_csv("../../data/sispea/indicateurs.csv", index_col=0)
indicator_df.head()

,code_ip,id_competence,name_competence,type_ip,theme,id_unit,symbol_unit,name_unit,format_ip,decimal_ip,min_warning_ip,max_warning_ip,min_critic_ip,max_critic_ip,validity_start,validity_end,libelle_grand_public
0,P205.3,2,assainissement collectif,Performance,Epuration,1,%,%,%f,0,0.0,100.0,0.0,100.0,NaN,NaN,Conformité de la performance des ouvrages d’ép...
1,P203.3,2,assainissement collectif,Performance,Collecte,1,%,%,%f,0,0.0,100.0,0.0,100.0,NaN,NaN,Conformité de la collecte des effluents aux pr...
2,P204.3,2,assainissement collectif,Performance,Epuration,1,%,%,%f,0,0.0,100.0,0.0,100.0,NaN,NaN,Conformité des équipements d’épuration aux pre...
3,P253.2,2,assainissement collectif,Performance,Reseau,1,%,%,%f,2,0.0,3.0,0.0,100.0,NaN,NaN,Renouvellement des réseaux de collecte des eau...
4,P103.2A,1,eau potable,Performance,Reseau,2,unité,unité,%d,0,0.0,100.0,0.0,100.0,NaN,2012.0,Connaissance et gestion patrimoniale des résea...


In [17]:
question_all_df2 = (question_all_df
                  .merge(indicator_df[["code_ip", "name_competence"]], left_on="indic", right_on="code_ip", how="left")
                  .drop("code_ip", axis=1)
                  .rename({"name_competence": "competence"}, axis=1)
)
question_all_df2.head()

,question,mot,indic,competence
0,Quel est le prix de service au m3 pour l'eau p...,prix,D102.0,eau potable
1,Quel est le prix du service au m3 pour l'assin...,prix,D204.0,assainissement collectif
2,Quelle est la connaissance du réseau en eau po...,connaissance,P103.2B,eau potable
3,Quel est le taux de connaissance du réseau en ...,connaissance,P202.2B,assainissement collectif
4,Quel est le taux de renouvellement des réseaux...,renouvellement,P107.2,eau potable


In [21]:
question_all_df2['competence'].unique().tolist()

['eau potable',
 'assainissement collectif',
 'assainissement non collectif',
 nan]

Save to csv

In [18]:
question_all_df2.to_csv("../../data/input/question_keyword_malou.csv")

### Format the question_keyword file if needed
Save the question-keyword file with "," as separator and an index column to be consistent with `question_keyword_malou.csv`

In [8]:
question_keyword_path = "../../data/input/question_keyword_v7.csv"
# To be run locally (not on the cloud)
# Read the CSV file with ";" as the separator
tmp_df = pd.read_csv(question_keyword_path, sep=';')
# Add an index column 
tmp_df.index.name = None
# Save the DataFrame 
tmp_df.to_csv(question_keyword_path, sep=',', index=True)

## Test the Page Finder

In [9]:
file_path = "../../data/input/pdfs/"
file_name = "RPQS_Corbel_cp73160_rpqsid_728273_AC_2022.pdf"

pdf_reader = PDFReader(file_path+file_name, 
                       extract_tables=True,
                       text_extraction_method="PyPDF2",
                       table_extraction_method="PDFPlumber")

pages = pdf_reader.textpages
all_tables = pdf_reader.dftables
toc_indices = pdf_reader.find_toc_pages()
print("toc_indices = ", toc_indices)

toc_indices =  []


Show the segmentation df

In [10]:
question_keyword_path = "../../data/input/question_keyword_v7.csv"

pagefinder = PageFinder(question_keyword_path, "assainissement collectif")
segmentation_df = pagefinder.get_segmentation_df(pages, all_tables, toc_indices).sort_values(by="question")

segmentation_df.sort_values(by="indicator")

,indicator,question,keyword_regex,relevant_pages,table_relevant_pages
1,D201.0,Quelle est la valeur de l'indicateur D201.0,\bD201.0s?\b,[],[]
0,D201.0,Quel le nombre d'habitants desservis par le ré...,\bhabitants?\b,[],NaN
2,D202.0,Quel est le nombre d'autorisations de déversem...,\bautorisations?\b,[],NaN
3,D202.0,Quelle est la valeur de l'indicateur D202.0,\bD202.0s?\b,[],[]
5,D203.0,Quelle est la valeur de l'indicateur D203.0,\bD203.0s?\b,[],[]
4,D203.0,Quelle est la quantité de boues évacuées (D203...,\bboues?\b,"[2, 6, 7]",NaN
6,D204.0,Quel est le prix au m3 du service d'assainisse...,\b120\s*m\s*3s?\b,[4],NaN
7,D204.0,Quelle est la valeur de l'indicateur D204.0,\bD204.0s?\b,[],[]
8,P201.1,Quel est le taux de desserte du réseau d'assai...,\bdessertes?\b,[1],NaN
9,P201.1,Quelle est la valeur de l'indicateur P201.1,\bP201.1s?\b,[],[]


In [7]:
table = all_tables[9][1]
table

,Col_0,Indicateurs descriptifs des services,Col_2
0,D201.0,Estimation du nombre d'habitants desservis par...,429
1,D202.0,Nombre d'autorisations de déversement d'efflue...,0
2,D203.0,Quantité de boues issues des ouvrages d'épuration,Sans objet
3,D204.0,Prix TTC du service au m3 pour 120 m3 (valeur ...,"1,43 € TTC"
4,None,Indicateurs de performance,None
5,P201.1,Taux de desserte par des réseaux de collecte d...,ND
6,P202.2,Indice de connaissance et de gestion patrimoni...,93
7,P203.3,Conformité de la collecte des effluents aux pr...,ND
8,P204.3,Conformité des équipements d’épuration aux pre...,Sans objet
9,P205.3,Conformité de la performance des ouvrages d’ép...,Sans objet
